<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_8_Resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this module, we will resample all of our products to a common pixel size. Since 4 of our 7 products already have a pixel size of 30 meters, we will use 30 meters as our shared pixel size.

To accomplish this, we will use the gdal.Warp method. gdal.Warp takes the following optional arguments (among others):
* xRes
* yRes
* resampleAlg

Since we want to create square pixels measuring 30 meters by 30 meters in size, we will set xRes and yRes to both equal 30. The resampleAlg argument determines how the input map will go about determining the pixel value for each output 30x30m pixel. There are several options for this argument. In this case, we will use 'mode', which is the Mode resampling algorithm. For more information on the different resampling algorithms available in GDAL, see [this page](https://gdal.org/en/stable/programs/gdalwarp.html) and scroll down to "-r resampling_method".

# Step 1: Import packages

In [ ]:
import ee
import geemap
from google.colab import drive
import os
import glob
from osgeo import gdal
import numpy as np
import pandas as pd
import time
from osgeo import gdal, osr

In [ ]:
ee.Authenticate()

ee.Initialize(project='servir-sco-assets')

# MODIFIABLE VARIABLE ALERT

In [ ]:
my_gee_folder = "users/mickymags/sep_arkansas/"
my_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/sep/sep_arkansas/"
other_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/Flood_Intercomparison/"
flood_event_desc = 'sep_arkansas'
time_id = '09092043'                # put in a string correlating to the current time which you are running the code -- will help with identifying exports

In [ ]:
aoi = ee.FeatureCollection(my_gee_folder + "aoi")
roi = aoi.geometry()
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

# Step 2: Resample GFM, MCDWD, and VFM products (and maybe HYDROSAR)

In order to run object extraction statistics, we must have all products in a common projection. Thus, we will use gdal.Warp to resample these products to 30 meters

In [ ]:
drive.mount('/content/drive/')

In [ ]:
os.chdir(my_Gdrive_folder)

In [ ]:
pwd

In [ ]:
ls

## Step 2 Part 1: GFM

In [ ]:
os.chdir('GFM')

In [ ]:
ls m*

In [ ]:
infile = glob.glob('harmonized_gfm*')[0]

In [ ]:
infile

In [ ]:
outfile = 'gfm_resampled_' + flood_event_desc + '.tif'

In [ ]:
info = gdal.Info(infile)
find = info.find('Data axis')
proj = info[find-8:find-3]
my_proj = 'EPSG:'+proj
print(my_proj)

In [ ]:
gdal.Warp(outfile, infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls gfm_re*

## Step 2 Part 2: MCDWD

In [ ]:
os.chdir('../MCDWD')

In [ ]:
pwd

In [ ]:
mcdwd_infile = infile = glob.glob('harmonized_mcd*')[0]

In [ ]:
mcdwd_infile

In [ ]:
ls m*

In [ ]:
mcdwd_outfile = 'mcdwd_intermediate_' + flood_event_desc + '.tif'

In [ ]:
gdal.Warp(mcdwd_outfile, mcdwd_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls mcdwd_re*

In [ ]:
inter_mcdwd = mcdwd_outfile
final_mcdwd = f'mcdwd_resampled_{flood_event_desc}.tif'
vector_mcdwd = glob.glob('drive_export*')[0]

In [ ]:
gdal.Warp(
    final_mcdwd,
    inter_mcdwd,
    cutlineDSName=vector_mcdwd,
    cropToCutline=True,
    dstNodata = 2
)

## Step 2 Part 3: VFM

In [ ]:
os.chdir('../VFM')

In [ ]:
pwd

In [ ]:
ls

In [ ]:
vfm_infile = infile = glob.glob('harmonized_vfm*')[0]

In [ ]:
vfm_outfile = 'vfm_intermediate_' + flood_event_desc + '.tif'

In [ ]:
vfm_infile

In [ ]:
gdal.Warp(vfm_outfile, vfm_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls v*

In [ ]:
inter_vfm = vfm_outfile
final_vfm = f'vfm_resampled_{flood_event_desc}.tif'
vector_mcdwd = glob.glob('drive_export*')[0]

In [ ]:
gdal.Warp(
    final_vfm,
    inter_vfm,
    cutlineDSName=vector_vfm,
    cropToCutline=True,
    dstNodata = 2
)

In [ ]:
pwd

## Step 2 Part 4: HYDROSAR

In some cases, HYDROSAR will not have a pixel size of 30 meters. If it does not, we will undergo the same workflow we did for VFM, GFM, and MCDWD. Let's check the pixel size of HYDROSAR

In [ ]:
os.chdir('../HYDROSAR')

In [ ]:
hydrosar_infile = infile = glob.glob('harmonized_hydrosar*')[0]

In [ ]:
hydrosar_file = gdal.Open(hydrosar_infile)

hydrosar_pixel_size = hydrosar_file.GetGeoTransform()[1]

if hydrosar_pixel_size == 30:
  print("HYDROSAR has a pixel size of 30 meters. Skip to step 3.")
else:
  print("HYDROSAR has a pixel size slightly different than 30 meters. Continue running the code below.")

In [ ]:
hydrosar_outfile = 'hydrosar_resampled_' + flood_event_desc + '.tif'

In [ ]:
gdal.Warp(hydrosar_outfile, hydrosar_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

# Step 3: Exporting to Google Earth Engine

In [ ]:
pwd

In [ ]:
hydrosar_str = glob.glob('harmonized*')[0]
hydrosar_fi = gdal.Open(hydrosar_str)
wkt_projection = hydrosar_fi.GetProjection()
spatial_ref = osr.SpatialReference(wkt_projection)
epsg_code = spatial_ref.GetAttrValue('AUTHORITY', 1)
proj = 'EPSG:' + epsg_code
print(proj)

In [ ]:
# Define a function that exports an Image to a Google Earth Engine Asset.
def exporter(img, asset_id):

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = proj,
                                  scale = 30,
                                  maxPixels = 1e13)
  return 0

In [ ]:
# Export VFM
vfm_resampled_harmonized_aid = my_gee_folder + 'vfm_resampled_harmonized'
exporter(vfm_resampled_final, vfm_resampled_harmonized_aid)

# Export GFM
gfm_resampled_harmonized_aid = my_gee_folder + 'gfm_resampled_harmonized'
exporter(gfm_resampled_final, gfm_resampled_harmonized_aid)

# Export MCDWD
mcdwd_resampled_harmonized_aid = my_gee_folder + 'mcdwd_resampled_harmonized'
exporter(mcdwd_resampled_final, mcdwd_resampled_harmonized_aid)

If you had to resample HYDROSAR